In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import time, csv, requests, os
from config import google_api_key
from config import visual_rapid_api_key
from natl_parks import us_natl_parks

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Lists for data cleaning
from city_states import format_us_states
from states_per_park import states_by_park

# Consolidate CSVs into one CSV
from names_of_csvs import all_csv

In [2]:
API_KEY = google_api_key

In [3]:
def get_coor(park, API_KEY):
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
    payload = {
        "key" : API_KEY,
        "input" : park,
        "inputtype" : "textquery",
        "fields" : "name,place_id,geometry/location,formatted_address"
    }
    r = requests.get(url, params = payload)

    if r.status_code == 200:
        return r.json()
    else:
        return None
    
all_park_info = []
for park in us_natl_parks:
    result = get_coor(park, API_KEY)
    if result:
        all_park_info.append(result)

In [4]:
latitude = []
longitude = []
park = []
states = []

for row in all_park_info:
    latitude.append(row["candidates"][0]["geometry"]["location"]["lat"])
    longitude.append(row["candidates"][0]["geometry"]["location"]["lng"])
    park.append(row['candidates'][0]['name'])
    states.append(row['candidates'][0]['formatted_address'])

In [5]:
df = pd.DataFrame({
    "Latitude":latitude,
    "Longitude":longitude,
    "Park": park
})

df.to_csv("Park Coordinates.csv", index = False)

In [6]:
API_KEY = visual_rapid_api_key

In [9]:
# Convert types, zip lat and long to create coordinates
lat_lngs = []
latitude = df["Latitude"].tolist()
longitude = df["Longitude"].tolist()
lat_lngs = zip(latitude, longitude)

# Identify nearest city for each coordinate
cities = []
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to our cities list
    if city not in cities:
        cities.append(city)

# Complete list where citipy unable to assign
cities.insert(8, "moab")
cities.insert(28, "pine springs")
cities.insert(38, "kiana")
cities.insert(58, "mccarthy")
cities.insert(59, "canyon village")
# Swap citipy British selection for US city
cities1 = ["coral bay" if x == "road town"  else x for x in cities]
# Swap citipy Mexico selection for US city
cities2 = ["terlingua" if x == "ojinaga"  else x for x in cities1]

# Concatenate Nearest City to State
concat_func = lambda x,y: x + "," + str(y)
nearest_city_and_state = list(map(concat_func, cities2, states_by_park))

# Use dictionary comprehension to convert lists to dictionary (to run loop through values)
states_dict = {states[i]: nearest_city_and_state[i] for i in range(len(states))}

#Create function to request historical weather by Nearest City and State
def hist_weather(value, API_KEY):
    f = open(str(value)+".csv","w+")
    
    url = "https://visual-crossing-weather.p.rapidapi.com/history"

    querystring = {
        "startDateTime":"2015-01-01T00:00:00",
        "aggregateHours":"24",
        "location": value,
        "endDateTime":"2019-12-30T24:00:00",
        "unitGroup":"us"
    }
    
    headers = {
        'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com",
        'x-rapidapi-key': API_KEY
        }

    r = requests.request("GET", url, headers = headers, params = querystring)

    if r:
        final = r.text
        f.write(final)
        f.close()
    else:
        print("No historical weather available for "+ str(value))
        f.close()

# Loop request for the Nearest City,State
for value in states_dict.values():   
    hist_weather(value, API_KEY)

In [34]:
# Compile CSVs into single CSV
with open("his_weather_final.csv", "w") as csv_file:
    fileWriter = csv.writer(csv_file)
    for place in all_csv:
        file_to_load = os.path.join(place)
        with open(file_to_load, "r") as weather_data:
            fileReader = csv.reader(weather_data, delimiter = ",")
            # Skip Headers: address, Date time, Cloud Cover, Latitude, Longitude, Resolved Address
            next(fileReader)
            for row in fileReader:
                fileWriter.writerow(row)